In [1]:
import numpy as np
from qokit.portfolio_optimization import get_problem

In [2]:
po_problem = get_problem(N=6,K=3,q=0.5,seed=1,pre=1)
means_in_spins = np.array([po_problem['means'][i] - po_problem['q'] * np.sum(po_problem['cov'][i, :]) for i in range(len(po_problem['means']))])
scale = 1 / (np.sqrt(np.mean((( po_problem['q']*po_problem['cov'])**2).flatten()))+np.sqrt(np.mean((means_in_spins**2).flatten())))

po_problem = get_problem(N=6,K=3,q=0.5,seed=1,pre=scale)

In [3]:
from qokit.portfolio_optimization import portfolio_brute_force, get_sk_ini
from qokit.qaoa_objective_portfolio import get_qaoa_portfolio_objective

In [19]:
p = 1
qaoa_obj = get_qaoa_portfolio_objective(po_problem=po_problem,p=p,ini='dicke',mixer='trotter_ring',T=1,simulator='python')
best_portfolio = portfolio_brute_force(po_problem,return_bitstring=False)

x0 = get_sk_ini(p=p) 

In [17]:
po_energy = qaoa_obj(x0).real
po_ar = (po_energy-best_portfolio[1])/(best_portfolio[0]-best_portfolio[1])
print(f"energy = {po_energy}, Approximation ratio = {po_ar}")

energy = 0.004962484517944696, Approximation ratio = 0.7564585339722991


In [20]:
from scipy.optimize import minimize
import nlopt

In [21]:
def minimize_nlopt(f, x0, rhobeg=None, p=None):
    def nlopt_wrapper(x, grad):
        if grad.size > 0:
            sys.exit("Shouldn't be calling a gradient!")
        return f(x).real

    opt = nlopt.opt(nlopt.LN_BOBYQA, 2 * p)
    opt.set_min_objective(nlopt_wrapper)

    opt.set_xtol_rel(1e-8)
    opt.set_ftol_rel(1e-8)
    opt.set_initial_step(rhobeg)
    xstar = opt.optimize(x0)
    minf = opt.last_optimum_value()

    return xstar, minf

In [ ]:
_, opt_energy = minimize_nlopt(qaoa_obj, x0, p=1, rhobeg=0.01/1)
opt_ar = (opt_energy-best_portfolio[1])/(best_portfolio[0]-best_portfolio[1])
print(f"energy = {opt_energy}, Approximation ratio = {opt_ar}")